In [ ]:
# Install Hugging Face Transformers and ROUGE
!pip install transformers datasets rouge-score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Install Libraries

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
from rouge_score import rouge_scorer

# Load Data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("allenai/scitldr", "AIC")  # Scientific paper dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

scitldr.py:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/7.23M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

In [ ]:
train_df = dataset['train']
test_df = dataset['test']
val_df = dataset['validation']

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 1992, Validation: 619, Test: 618


In [ ]:
#train_df = df
#val_df = dfval
#test_df = dftest

# train_df = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/train.jsonl", lines=True)
# val_df = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/dev.jsonl", lines=True)
# test_df = pd.read_json("/content/drive/MyDrive/Binus/Sem 5/textmining/test.jsonl", lines=True)

# print(f"Training size: {len(train_df)}, Validation size: {len(val_df)}, Testing size: {len(test_df)}")


Training size: 1992, Validation size: 619, Testing size: 618


In [ ]:
# class SummarizationDataset(Dataset): # class Cleansing yang final
#     def __init__(self, dataframe, tokenizer, max_input_len=512, max_output_len=128):
#         self.data = dataframe
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         row = self.data.iloc[idx]
#         #source = row["text"]
#         source = "summarize: " + " ".join(row["text"])
#         target = row["target"]

#         source_encodings = self.tokenizer(
#             source,
#             max_length=self.max_input_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )
#         target_encodings = self.tokenizer(
#             target,
#             max_length=self.max_output_len,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": source_encodings["input_ids"].squeeze(),
#             "attention_mask": source_encodings["attention_mask"].squeeze(),
#             "labels": target_encodings["input_ids"].squeeze(),
#         }


In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class SCITldrDataset(Dataset): # class buat NonCleansing yang final
#     def __init__(self, dataset, tokenizer, max_input_len=512, max_output_len=128):
#         self.dataset = dataset  # Save the dataset object directly
#         self.tokenizer = tokenizer
#         self.max_input_len = max_input_len
#         self.max_output_len = max_output_len

#     def __len__(self):
#         return len(self.dataset)  # Use the length of the dataset

#     def __getitem__(self, idx):
#         # Access the data entry using .iloc to ensure integer-based indexing
#         item = self.dataset.iloc[idx]
#         # Join the list of strings in 'source' into a single string
#         input_text = "summarize: " + " ".join(item["source"])
#         target_text = item["target"]

#         # Tokenize input and output
#         inputs = self.tokenizer(
#             input_text,
#             max_length=self.max_input_len,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )
#         targets = self.tokenizer(
#             target_text,
#             max_length=self.max_output_len,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": inputs["input_ids"].squeeze(),
#             "attention_mask": inputs["attention_mask"].squeeze(),
#             "labels": targets["input_ids"].squeeze(),
#         }

In [ ]:
class SCITldrDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.dataset = dataset  # Save the dataset object directly
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)  # Use the length of the dataset

    def __getitem__(self, idx):
        item = self.dataset[idx]  # Access an individual data entry
        # Join the list of strings in 'source' into a single string
        input_text = "summarize: " + " ".join(item["source"])
        target_text = item["target"][0]

        # Tokenize input and output
        inputs = self.tokenizer(
            input_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        targets = self.tokenizer(
            target_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze(),
        }

# Set Model

In [ ]:
model_name = "t5-small"  # Change to 't5-base' or larger if needed
#model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using device: cuda


In [ ]:
# Prepare datasets
train_dataset = SCITldrDataset(train_df, tokenizer)  # pake yang Noncleansing karena nama collumnnya beda
val_dataset = SCITldrDataset(val_df, tokenizer)
test_dataset = SCITldrDataset(test_df, tokenizer)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Model Training arc

In [ ]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5) # perlu diganti adamw nya

epochs = 10  # Maximum number of epochs
patience = 3  # Stop if no improvement in 'patience' epochs
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"Epoch {epoch + 1}/{epochs} starts...")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print loss every 10 steps
        if (step + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Step {step + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    # Print average loss for the epoch
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} completed. Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}")

    # Check for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pt")  # Save the best model
        print("Best model saved.")
    else:
        patience_counter += 1
        print(f"No improvement in validation loss. Patience counter: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10 starts...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1, Step 10/249, Loss: 7.3935
Epoch 1, Step 20/249, Loss: 4.5893
Epoch 1, Step 30/249, Loss: 1.9881
Epoch 1, Step 40/249, Loss: 1.5706
Epoch 1, Step 50/249, Loss: 1.4734
Epoch 1, Step 60/249, Loss: 1.3990
Epoch 1, Step 70/249, Loss: 1.3135
Epoch 1, Step 80/249, Loss: 1.6430
Epoch 1, Step 90/249, Loss: 1.1995
Epoch 1, Step 100/249, Loss: 1.0077
Epoch 1, Step 110/249, Loss: 1.0271
Epoch 1, Step 120/249, Loss: 1.0856
Epoch 1, Step 130/249, Loss: 1.1488
Epoch 1, Step 140/249, Loss: 0.8736
Epoch 1, Step 150/249, Loss: 0.9947
Epoch 1, Step 160/249, Loss: 0.8912
Epoch 1, Step 170/249, Loss: 0.9165
Epoch 1, Step 180/249, Loss: 0.8268
Epoch 1, Step 190/249, Loss: 1.0765
Epoch 1, Step 200/249, Loss: 1.0591
Epoch 1, Step 210/249, Loss: 1.0135
Epoch 1, Step 220/249, Loss: 0.8481
Epoch 1, Step 230/249, Loss: 0.9432
Epoch 1, Step 240/249, Loss: 0.8541
Epoch 1/10 completed. Training Loss: 1.7372
Validation Loss: 0.8053
Best model saved.
Epoch 2/10 starts...
Epoch 2, Step 10/249, Loss: 0.8488
Epo

# Evaluation Model

In [ ]:
# Load the best model
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

predictions = []
references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_refs = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

        predictions.extend(decoded_preds)
        references.extend(decoded_refs)

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

rouge1, rouge2, rougeL = 0, 0, 0
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge1 += scores["rouge1"].fmeasure
    rouge2 += scores["rouge2"].fmeasure
    rougeL += scores["rougeL"].fmeasure

n = len(predictions)
print(f"ROUGE-1: {rouge1 / n:.4f}")
print(f"ROUGE-2: {rouge2 / n:.4f}")
print(f"ROUGE-L: {rougeL / n:.4f}")

<ipython-input-9-5a2e5aa9e4ab>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))


ROUGE-1: 0.3642
ROUGE-2: 0.1610
ROUGE-L: 0.3034


In [ ]:
print(f"ROUGE-1: {rouge1 * 100 / n:.4f} %")
print(f"ROUGE-2: {rouge2 * 100 / n:.4f} %")
print(f"ROUGE-L: {rougeL * 100 / n:.4f} %")

ROUGE-1: 36.4185 %
ROUGE-2: 16.0994 %
ROUGE-L: 30.3433 %
